# 패키지 임포트

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import nltk
import os
import string
import re
import json
import keras
from data_preprocessing import *
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# 데이터 로드

In [3]:
train_df = pd.read_csv('C:/Users/user/Desktop/bilm-tf-master/ag_news_dataset/train.csv')
test_df = pd.read_csv('C:/Users/user/Desktop/bilm-tf-master/ag_news_dataset/test.csv')

# 전처리

In [4]:
train_df = combine_title_and_description(train_df)
test_df = combine_title_and_description(test_df)

#각 클래스별로 5000개씩 총 2만개의 데이터를 샘플랭(너무 크면 TextCuboid의 용량이 너무 커진다)
sampled_df = train_df.groupby("Class Index").apply(lambda x: x.sample(5000, random_state=10))

#전처리 특수기호 없애기
sampled_df['text']=sampled_df['text'].apply(clean_text)

sampled_df = sampled_df.reset_index(drop=True)

train_df = sampled_df.groupby("Class Index").apply(lambda x: x.sample(4000, random_state=10))
train_idx = [x[1] for x in train_df.index]
test_df = sampled_df.drop(train_idx)

x_train=list(train_df['text'])
y_train=list(train_df['Class Index'])
x_test=list(test_df['text'])
y_test=list(test_df['Class Index'])

to_txt_filter=x_train+x_test
y=list(y_train)+list(y_test)

In [5]:
encoder=LabelEncoder()

encoder.fit(y)

label=encoder.transform(y)

y_train=list(label[:16000])
y_test=list(label[16000:])

In [6]:
#불용어 불러오기
with open('C:/Users/user/Desktop/english.txt', 'r', encoding='utf-8') as file:
    stopwords = [line.strip() for line in file]

In [7]:
stopwords=stopwords+['aa','aaa', 'ab']

In [9]:
#문서 분류에 도움되는 10,000개 단어 선별
vect = CountVectorizer(stop_words=stopwords)
X_dtm = vect.fit_transform(to_txt_filter)
X_dtm = X_dtm.toarray()
X_new = SelectKBest(chi2, k=10000).fit(X_dtm, y)
TorF = X_new.get_support()
TorF
import numpy as np
word_view=np.array(vect.get_feature_names_out())
feature_lst10000=word_view[TorF]

C:\ProgramData\anaconda3\envs\class\lib\site-packages\sklearn\feature_extraction\text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ain', 'daren', 'hadn', 'herse', 'himse', 'itse', 'mayn', 'mightn', 'mon', 'mustn', 'myse', 'needn', 'oughtn', 'shan'] not in stop_words.
  warnings.warn(


# tensor space model

In [10]:
doc_lst=[]
for sen in x_train:
    doc_lst.append(sen.split())

In [11]:
test_lst=[]
for sen in x_test:
    test_lst.append(sen.split())

In [13]:
#Train_data에서 문서가 갖고 있는 선별한 feauture의 수 확인
count_lst=[]
for i in range(16000):
    total_feature_cnt=0
    for j in range(10000):
        if feature_lst10000[j] in doc_lst[i]:
            total_feature_cnt+=1
    count_lst.append(total_feature_cnt)
    
print('Train_data에서 가장 많은 feature를 가진 문서의 경우 feature',max(count_lst),' 개를 가짐')
print('Train_data에서 가장 적은 feature를 가진 문서의 경우 feature',min(count_lst),' 개를 가짐')

train_max_feature=max(count_lst)

Train_data에서 가장 많은 feature를 가진 문서의 경우 feature 46  개를 가짐
Train_data에서 가장 적은 feature를 가진 문서의 경우 feature 1  개를 가짐


In [14]:
#Test_data에서 문서가 갖고 있는 선별한 feauture의 수 확인
count_lst=[]
for i in range(4000):
    
    total_feature_cnt=0
    for j in range(10000):
        if feature_lst10000[j] in test_lst[i]:
            total_feature_cnt+=1
    count_lst.append(total_feature_cnt)
    
print('Test_data에서 가장 많은 feature를 가진 문서의 경우 feature',max(count_lst),' 개를 가짐')
print('Test_data에서 가장 적은 feature를 가진 문서의 경우 feature',min(count_lst),' 개를 가짐')

test_max_feature=max(count_lst)

max_feature=max(train_max_feature,test_max_feature)

Test_data에서 가장 많은 feature를 가진 문서의 경우 feature 40  개를 가짐
Test_data에서 가장 적은 feature를 가진 문서의 경우 feature 1  개를 가짐


In [15]:
#1-Channel TextCuboid 생성
textcuboid=[]
for i in range(16000):
    frame1=np.zeros((max_feature,256))
    idx_cnt=0
    for j in range(10000):
        if feature_lst10000[j] in doc_lst[i]:
            #문서에서 선별한 단어(feature)의 위치를 찾아 임베딩 벡터 추출
            frame1[idx_cnt]=np.load('./elmo_embedding/train(ag256)/doc%d.npy'%i)[doc_lst[i].index(feature_lst10000[j])]
            idx_cnt+=1
    textcuboid.append(frame1)

textcuboid=np.array(textcuboid)

np.save('./1-Channel textcuboid_ag(elmo).npy',textcuboid)

In [16]:
#1-Channel TextCuboid 생성
textcuboid_test=[]

for i in range(4000):
    frame1=np.zeros((max_feature,256))
    idx_cnt=0
    for j in range(10000):
        if feature_lst10000[j] in test_lst[i]:
            frame1[idx_cnt]=np.load('./elmo_embedding/test(ag256)/test%d.npy'%i)[test_lst[i].index(feature_lst10000[j])]
            idx_cnt+=1
    textcuboid_test.append(frame1)

textcuboid_test=np.array(textcuboid_test)
np.save('./1-Channel textcuboid_test_ag(elmo).npy',textcuboid_test)